In [3]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ----------------------------- ---------- 1.0/1.4 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 5.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd # type: ignore
import lightgbm as lgb  # type: ignore
import warnings
warnings.filterwarnings("ignore")


train = pd.read_excel('data/UNSW_NB15_training.xlsx')
test = pd.read_excel('data/UNSW_NB15_testing.xlsx')

print(lgb.__version__)

4.5.0


In [16]:
features = ['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 
            'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'swin', 'dwin', 
            'is_sm_ips_ports', 'attack_cat']
target = 'attack_cat_number'  # Adjust if your target column has a different name

# Identify categorical features
cat_features = ['proto', 'service', 'state', 'attack_cat']
for col in cat_features:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Split into X (features) and y (labels)
X_train, y_train = train_df[features], train_df[target]
X_test, y_test = test_df[features], test_df[target]

KeyError: 'attack_cat_number'

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_features)

# Define model parameters
params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'num_class': 10,
    'verbose': -1
}

# Train model
model = lgb.train(
    params,
    train_data,
    num_boost_round=200,  # Ensure this is before early stopping
    valid_sets=[test_data],
    valid_names=["validation"],  # Give validation set a name
    # early_stopping_rounds=50  # This should be placed after valid_sets
)


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: attack_cat: object